# Slime Mould Algorithm Simulation in Python

## Overview

This Jupyter Notebook presents a simulation of a Slime Mould Algorithm (SMA) using Python. The primary objective of this simulation is to implement this metaheuristic in a natural setting (agents searching for food). Initially, I contemplated employing compute shaders for this simulation to harness the power of parallel computation and produce nice visuals. However, to maintain simplicity and accessibility, I opted for Matplotlib, a widely used Python library for data visualization.

The SMA simulation is built on several core concepts:

- **Agent Movement**: Agents move towards food sources based on a calculated 'smell' or odor map.
- **Oscillation**: Agents' velocity changes periodically, to model state switching.
- **Odor Map**: This map represents the intensity of food odors across the grid, guiding agents towards food sources.
- **Dynamic Behavior**: Agent behavior and movement are influenced by various factors, including their position, the odor map, and an adjustable parameter `A_param`.

The simulation uses NumPy for efficient numerical computations and Matplotlib for visualizing the simulation steps. Additionally, interactive widgets provided by `ipywidgets` allow users to customize the simulation parameters like the number of agents, food sources, grid size, and simulation steps.

In [68]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### Function: initialize_simulation

#### Purpose
The `initialize_simulation` function sets up the initial conditions for the Slime Mould Algorithm (SMA) simulation. It initializes agents, food sources, and now includes the initialization of a trace grid to record the movement paths of agents.

#### Parameters
- `grid_size`: The size of the simulation grid.
- `num_agents`: The number of agents (slime mould entities) to simulate.
- `num_food_sources`: The number of food sources on the grid.
- `start_in_middle`: A boolean flag to start agents in the middle of the grid or distribute them randomly.

#### Functionality
1. **Agent and Food Source Initialization**: Initializes agents and food sources on the grid.
2. **Trace Grid Initialization**: Creates a `trace_grid` as a NumPy array of zeros, with the same dimensions as the simulation grid. This grid will be used to record the agents' movement paths.

#### Returns
- `agents`: A list of agent positions.
- `food_sources`: A list of food source positions.
- `trace_grid`: A grid to track the agents' paths.

#### Modifications for Tracing
- This function now initializes an additional grid, `trace_grid`, which is used to record the trails left by agents as they move across the grid.

The addition of the trace grid allows for a more dynamic and visually informative simulation, providing insights into the movement patterns of agents over time.

In [69]:
# Function to initialize the grid, agents, and food sources
def initialize_simulation(grid_size, num_agents, num_food_sources, start_in_middle):
    agents = initialize_agents(grid_size, num_agents, start_in_middle)
    trace_grid = np.zeros((grid_size, grid_size))
    food_sources = [(random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)) for _ in range(num_food_sources)]
    return agents, food_sources, trace_grid

### Function: initialize_agents

#### Purpose
`initialize_agents` is responsible for placing agents (representing the slime mould) on the simulation grid at the start of the simulation. The function provides an option to either place these agents randomly across the grid or cluster them near the middle.

#### Parameters
- `grid_size`: An integer representing the dimensions of the square simulation grid (`grid_size x grid_size`).
- `num_agents`: The number of agents to be initialized on the grid.
- `start_in_middle`: A boolean flag. When `True`, agents are initialized near the center of the grid. If `False`, they are placed randomly across the entire grid.

#### Functionality
1. **Centralized Initialization (if `start_in_middle` is `True`)**:
   - Determines the mid-point of the grid.
   - Initializes agents within a small range around this mid-point, creating a localized cluster of agents.
2. **Random Initialization (if `start_in_middle` is `False`)**:
   - Places each agent at a random location on the grid.
   - Ensures a diverse and scattered initial distribution of agents.

#### Returns
- `agents`: A list of tuples, with each tuple representing the coordinates (x, y) of an agent on the grid. This list forms the initial population of agents for the simulation.

This function sets the initial conditions for the agent distribution, a critical aspect influencing the dynamics of the ensuing simulation. The choice of centralized or random initialization can significantly impact the behavior and efficiency of agents in locating food sources.

In [70]:
# Function to initialize agents on the grid
def initialize_agents(grid_size, num_agents, start_in_middle):
    if start_in_middle:
        mid_point = grid_size // 2
        area_range = 3
        agents = [(random.randint(mid_point - area_range, mid_point + area_range),
                   random.randint(mid_point - area_range, mid_point + area_range)) for _ in range(num_agents)]
    else:
        agents = [(random.randint(0, grid_size - 1), random.randint(0, grid_size - 1)) for _ in range(num_agents)]
    return agents

### Function: create_odor_map

#### Purpose
The `create_odor_map` function generates a map representing the distribution of odors (from food sources) across the simulation grid. This map is crucial for guiding agents towards food sources.

#### Parameters
- `food_sources`: A list of tuples, each representing the coordinates of a food source on the grid.
- `grid_size`: An integer specifying the size of the grid. The grid is square, with dimensions `grid_size x grid_size`.

#### Functionality
1. **Grid Coordinate Creation**: Generates a grid of coordinates using `numpy.meshgrid`, creating a two-dimensional grid where each cell represents a point in the simulation space.
2. **Food Source Array Conversion**: Converts the list of food source coordinates into a NumPy array for efficient computation.
3. **Distance Calculation**: Calculates the Euclidean distance from each grid cell to each food source in a vectorized manner using NumPy. This step is crucial for determining the intensity of the odor at each point on the grid.
4. **Minimum Distance Determination**: Identifies the minimum distance from each grid cell to the nearest food source. This step is key in representing the strongest odor perceived at each point.
5. **Odor Map Calculation**: Calculates the odor strength as the inverse of the distance, ensuring that closer food sources have a stronger odor. A constant is added to the distance to prevent division by zero.

#### Returns
- `odor_map`: A 2D NumPy array representing the odor strength at each point on the grid. Higher values indicate stronger odors, guiding agents more effectively towards food sources.

This function is integral to the simulation, as it provides the sensory landscape that agents use to navigate towards food sources, mimicking the behavior of real-world slime moulds in response to chemical gradients.

In [71]:
def create_odor_map(food_sources, grid_size):
    # Create a grid of coordinates
    x_coords, y_coords = np.meshgrid(np.arange(grid_size), np.arange(grid_size), indexing='ij')

    # Convert food_sources to a NumPy array for efficient operations
    food_sources_array = np.array(food_sources)

    # Calculate distances from each grid cell to each food source in a vectorized manner
    distances = np.sqrt((x_coords[:, :, np.newaxis] - food_sources_array[:, 0])**2 +
                        (y_coords[:, :, np.newaxis] - food_sources_array[:, 1])**2)

    # Find the minimum distance to a food source for each grid cell
    min_distances = distances.min(axis=2)

    # Calculate the odor strength as the inverse of the distance
    odor_map = 1 / (1 + min_distances)  # Adding 1 to avoid division by zero

    return odor_map

### Function: calculate_smell_index

#### Purpose
The `calculate_smell_index` function evaluates and ranks each agent based on their proximity to food sources, as indicated by the odor strength they detect. This ranking helps in simulating the behavior of agents as they navigate towards food sources.

#### Parameters
- `agents`: A list of tuples, each representing the coordinates (x, y) of an agent on the grid.
- `odor_map`: A 2D NumPy array where each element represents the odor strength at that point on the grid. Higher values indicate stronger odors.

#### Functionality
1. **Odor Strength Calculation**: Determines the odor strength at each agent's location by accessing the corresponding values from the `odor_map`.
2. **Smell Index Computation**: Ranks the agents based on the odor strength they experience. This is achieved using `numpy.argsort`, which returns the indices that would sort the agents based on the detected odor strength.

#### Returns
- `smell_index`: An array of indices that sorts agents in ascending order of the odor strength they detect. Agents detecting stronger odors (closer to food sources) will have higher ranks.

This function is crucial for simulating the decision-making process of agents as they navigate the grid. By prioritizing movement towards stronger odors, it mimics a real-world search strategy employed by organisms like slime moulds.


In [72]:
# Function to calculate the smell index for each agent
def calculate_smell_index(agents, odor_map):
    agent_smell_strengths = [odor_map[agent[0], agent[1]] for agent in agents]
    smell_index = np.argsort(agent_smell_strengths)  # Sort agents based on their smell strength
    return smell_index

### Function: update_vc

#### Purpose
`update_vc` (Variable Component Update) is designed to dynamically adjust a key parameter in the simulation over time. This function calculates a variable component (`vc`) for each simulation step, influencing the agents' movement and decision-making processes. The variable component is a combination of a linearly decreasing term and an oscillatory term.

#### Parameters
- `step`: The current simulation step, an integer.
- `max_steps`: The total number of steps in the simulation, an integer.
- `A_PARAM`: A constant parameter that scales the amplitude of the oscillation component.

#### Functionality
1. **Linear Component Calculation**: Computes a linearly decreasing value based on the current step and the total number of steps.
2. **Oscillation Component Calculation**: Calculates an oscillatory term using a sine function, scaled by `A_PARAM`. This introduces a periodic variation to the variable component.
3. **Combination and Clipping**: Adds the linear and oscillatory components to form `vc`, and then clips its value within the range `[-A_PARAM, A_PARAM]` to maintain controlled variability.

#### Returns
- `vc`: The computed variable component for the given step. This value is used in the simulation to modulate agent behaviors dynamically, introducing complexity and realism to their movement patterns.

This function plays a critical role in adding dynamism to the simulation, as it allows the behavior of agents to evolve over time, rather than remaining static or purely deterministic.

In [73]:
# Function to update the variable component (vc) based on the simulation step
def update_vc(step, max_steps, A_PARAM):
    linear_component = 1 - (step / max_steps)
    oscillation_component = A_PARAM * np.sin(np.pi * step / max_steps)
    vc = linear_component + oscillation_component
    vc = np.clip(vc, -A_PARAM, A_PARAM)
    return vc

### Function: approach_food_SMA

#### Purpose
`approach_food_SMA` (Slime Mould Algorithm for Approaching Food) is the core function that models the behavior of agents as they move towards food sources. This function implements the key mechanics of the SMA, taking into account various factors such as the odor map, the agents' relative positions, and the dynamic variable component to simulate realistic movement towards food.

#### Parameters
- `agents`: A list of tuples representing the current positions of the agents on the grid.
- `food_sources`: A list of tuples indicating the positions of food sources on the grid.
- `odor_map`: A 2D NumPy array representing the odor strength at each grid point.
- `smell_index`: An array of indices ranking the agents based on the odor strength at their current positions.
- `step`: The current step in the simulation.
- `max_steps`: The total number of steps in the simulation.
- `A_PARAM`: A parameter influencing the amplitude of the oscillatory component in agent movement.
- `GRID_SIZE`: The size of the simulation grid.

#### Functionality
1. **Variable Component Calculation**: Computes the variable component (`vc`) for the current step using `update_vc`.21. **Agent Movement**: Moves each agent towards the nearest food source based on calculated factors like odor strength and agent behavior parameters.3
2. **Trace Updating**: Increases the trace intensity on the `trace_grid` at each agent's current position4
3. **Trace Decay**: Applies a decay to the `trace_grid`, reducing the intensity of the trace over time to simulate a fading trail.

#### Returns
- `new_agents`: The updated positions of the agents.
- `trace_grid`: The updated grid containing the agents' trails.

This function is integral to the simulation, as it encapsulates the decision-making process and movement strategy of the agents, thereby driving the overall dynamics of the SMA simulation.

In [100]:
# Function for the Slime Mould Algorithm (SMA) model to approach food
def approach_food_SMA(agents, food_sources, odor_map, smell_index, step, max_steps, A_PARAM, GRID_SIZE, trace_grid, decay_rate=0.85):
    new_agents = []
    vc = update_vc(step, max_steps, A_PARAM)

    for idx, agent in enumerate(agents):
        W = calculate_weight(smell_index, idx, len(agents))
        closest_food = min(food_sources, key=lambda x: np.linalg.norm(np.array(x) - np.array(agent)))
        direction = np.array(closest_food) - np.array(agent)
        distance = np.linalg.norm(direction)

        vb = random.uniform(-A_PARAM, A_PARAM) * W
        new_position = determine_movement(agent, vb, vc, direction, distance, GRID_SIZE)

        # Update the trace grid with the current position of the agent
        trace_grid[agent] += 1  # Increase the trace intensity

        new_agents.append(new_position)

    # Decay the trace intensity over time
    trace_grid *= decay_rate
    return new_agents, trace_grid

### Function: calculate_weight

#### Purpose
`calculate_weight` is a supporting function in the Slime Mould Algorithm (SMA) simulation. It calculates a weight for each agent based on its position in the smell index. This weight influences the agent's movement, representing the decision-making process based on the perception of food odors.

#### Parameters
- `smell_index`: An array of indices that rank agents based on the strength of the odor they detect.
- `idx`: The index of the current agent for which the weight is being calculated.
- `num_agents`: The total number of agents in the simulation.

#### Functionality
1. **Weight Calculation**: Determines the weight (`W`) for an agent based on its rank in the smell index.
   - If the agent's rank is in the top half (i.e., it detects a stronger odor), `W` is calculated as `1 + log(smell_index[idx] + 1)`.
   - If the agent's rank is in the bottom half (i.e., it detects a weaker odor), `W` is calculated as `1 - log(smell_index[idx] + 1)`.
2. **Logarithmic Scaling**: The use of a logarithmic function introduces a non-linear scaling, making the weight more sensitive to changes in the agent's position within the smell index.

#### Returns
- `W`: The calculated weight for the agent. This weight plays a crucial role in modulating the agent's movement, particularly in how it biases the random component of the agent's step.

This function adds a layer of complexity to the simulation, as it ensures that agents closer to food sources (with a higher rank in the smell index) behave differently from those farther away, mimicking a realistic biological response to sensory stimuli.

In [75]:
# Function to calculate the weight of the slime mould based on smell index
def calculate_weight(smell_index, idx, num_agents):
    if smell_index[idx] < num_agents / 2:
        W = 1 + np.log(smell_index[idx] + 1)
    else:
        W = 1 - np.log(smell_index[idx] + 1)
    return W

### Function: determine_movement

#### Purpose
The `determine_movement` function calculates the new position of an agent in the Slime Mould Algorithm (SMA) simulation. This function takes into account various factors such as the agent's current position, direction towards the food source, and the variable components influencing movement.

#### Parameters
- `agent`: A tuple representing the current position (x, y) of the agent on the grid.
- `vb`: A variable bias component, influencing random movement of the agent.
- `vc`: A variable component, dynamically calculated for each simulation step, affecting directed movement.
- `direction`: A NumPy array representing the direction vector from the agent towards the closest food source.
- `distance`: The distance from the agent to the closest food source.
- `GRID_SIZE`: The size of the simulation grid.

#### Functionality
1. **Movement Calculation**:
   - If the distance to the food source is less than half the grid size, the agent's movement is primarily influenced by `vb`, simulating a random search.
   - If the distance is greater or equal to half the grid size, the movement is primarily directed by `vc`, simulating a more targeted approach towards the food source.
2. **Position Update**: The agent's position is updated based on the calculated movement.
3. **Boundary Check**: Ensures that the new position does not exceed the boundaries of the grid. This is achieved using `np.clip` to keep the agent's position within the limits of the grid.

#### Returns
- A tuple representing the new position (x, y) of the agent on the grid.

This function is a key component in the SMA simulation, dictating the movement strategy of each agent. It balances random exploration with directed movement towards food sources, reflecting the adaptive behavior of real-world slime moulds in response to environmental stimuli.

In [76]:
# Function to determine the new position of an agent
def determine_movement(agent, vb, vc, direction, distance, GRID_SIZE):
    if distance < GRID_SIZE / 2:
        new_position = np.array(agent) + vb * direction
    else:
        new_position = np.array(agent) + vc * direction
    new_position = np.clip(new_position, 0, GRID_SIZE - 1)
    return tuple(new_position.astype(int))

### Function: simulate_SMA

#### Purpose
`simulate_SMA` is the main function that orchestrates the Slime Mould Algorithm (SMA) simulation. It integrates various components of the simulation, such as initializing agents and food sources, calculating odor maps, and updating agent positions. The function also handles the visualization of the simulation process.

#### Parameters
- `grid_size`: The size of the simulation grid.
- `num_agents`: The number of agents (slime mould entities) to simulate.
- `num_food_sources`: The number of food sources to place on the grid.
- `steps`: The total number of steps for the simulation.
- `A_param`: A parameter that influences the amplitude of oscillatory components in agent movement.
- `start_in_middle`: A boolean flag determining whether agents start in the middle of the grid or randomly distributed.

#### Functionality
1. **Initialization**: Sets up the initial state of the simulation, including agents and food sources.
2. **Odor Map Generation**: Creates an odor map based on the initial food sources.
3. **Simulation Loop**: For each step in the simulation:
   - Calculates the smell index for agents based on the current odor map.
   - Updates the positions of the agents using `approach_food_SMA`.
   - Updates the odor map if food sources are consumed.
   - Visualizes the current state of the simulation.
   - Checks if all food sources are consumed to potentially end the simulation early.
4. **Visualization**: Utilizes Matplotlib to display the simulation grid, agents, food sources, and odor map.

#### Visualization
The function generates two plots:
- **Agents and Food Sources**: Shows the current positions of agents and food sources on the grid.
- **Odor Map**: Displays the odor intensity across the grid.

#### Closing the Simulation
Upon completion or if all food sources are consumed, the function closes the Matplotlib figure to prevent additional rendering.

This function acts as the central hub of the SMA simulation, bringing together all components and steps necessary to simulate the behavior of slime mould in a computational environment.

In [77]:
# Function to run the slime mould simulation
def simulate_SMA(grid_size, num_agents, num_food_sources, steps, A_param, start_in_middle):
    global agents, food_sources
    agents, food_sources, trace_grid = initialize_simulation(grid_size, num_agents, num_food_sources, start_in_middle)
    odor_map = create_odor_map(food_sources, grid_size) if food_sources else np.zeros((grid_size, grid_size))

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    for step in range(steps):
        if food_sources:
            smell_index = calculate_smell_index(agents, odor_map)
            agents, trace_grid = approach_food_SMA(agents, food_sources, odor_map, smell_index, step, steps, A_param, grid_size, trace_grid)
            odor_map = update_food_sources(food_sources, agents, odor_map, grid_size)

        display_simulation_step(axs, fig, agents, food_sources, odor_map, trace_grid, step, grid_size)  # Pass 'fig' as an argument

        if not food_sources:
            print("All food sources have been consumed. Simulation ended.")
            break

    plt.close(fig)

### Function: update_food_sources

#### Purpose
`update_food_sources` manages the state of food sources in the Slime Mould Algorithm (SMA) simulation. It updates the food sources based on their consumption by agents and accordingly recalculates the odor map to reflect the changes in the environment.

#### Parameters
- `food_sources`: A list of tuples representing the coordinates of food sources on the grid.
- `agents`: A list of tuples indicating the positions of agents on the grid.
- `odor_map`: A 2D NumPy array representing the current odor strength at each grid point.
- `grid_size`: The size of the simulation grid.

#### Functionality
1. **Food Consumption Check**: Iterates through the agents to check if any agent is at the same location as a food source. If so, it removes the food source from the list, indicating consumption.
2. **Odor Map Update**: If any food source is consumed, the function recalculates the odor map to reflect the new distribution of food sources. This is achieved by calling `create_odor_map`.
3. **State Management**: Keeps track of whether any food source has been consumed during the iteration.

#### Returns
- `odor_map`: An updated 2D NumPy array representing the new odor map after food sources have been consumed or the same odor map if no changes occurred.

This function is crucial for maintaining the dynamic nature of the simulation environment. It ensures that the agents' behavior and the odor landscape are consistently aligned with the current state of food sources, enhancing the realism of the simulation.

In [78]:
# Function to update the food sources after each simulation step
def update_food_sources(food_sources, agents, odor_map, grid_size):
    food_eaten = False
    for agent in agents:
        if agent in food_sources:
            food_sources.remove(agent)
            food_eaten = True
    if food_eaten and food_sources:
        odor_map = create_odor_map(food_sources, grid_size)
    return odor_map

### Function: display_simulation_step

#### Purpose
`display_simulation_step` visualizes the current state of the Slime Mould Algorithm (SMA) simulation, including the positions of agents, food sources, the odor map, and now the traces left by the agents.

#### Parameters
- `axs`: An array of Matplotlib axes for plotting the simulation state.
- `fig`: The Matplotlib figure object that contains the axes.
- `agents`: A list of tuples representing the positions of agents.
- `food_sources`: A list of tuples indicating the positions of food sources.
- `odor_map`: A 2D NumPy array representing the odor strength at each grid point.
- `trace_grid`: A 2D NumPy array representing the traces left by agents.
- `step`: The current step of the simulation.
- `GRID_SIZE`: The size of the simulation grid.

#### Functionality
1. **Clearing Axes**: Clears the current contents of the axes for new visualizations.
2. **Creating Display Grid**: Combines the positions of agents, food sources, and the traces into a single grid for display.
   - Marks agents and food sources with distinct values.
   - Adds the trace grid to the display grid.
3. **Visualization**:
   - The first subplot (`axs[0]`) displays the agents, food sources, and their traces.
   - The second subplot (`axs[1]`) shows the odor map.
   - Uses different color mappings for visual distinction and clarity.
4. **Updating Titles and Display**: Sets titles for each subplot and displays the updated figure.
5. **Clearing Output**: Clears the output of the previous step to ensure that only the w of the simulation dynamics.

This function is crucial for real-time monitoring and analysis of the SMA simulation, offering an intuitive and visually rich representation of the complex interactions and behaviors within the simulated environment.


In [98]:
def display_simulation_step(axs, fig, agents, food_sources, odor_map, trace_grid, step, GRID_SIZE):
    axs[0].clear()
    axs[1].clear()

    # Create a display grid for agents, food, and traces
    display_grid = np.clip(trace_grid, 0, 1)  # Ensure trace values are between 0 and 1

    # Mark agents with a specific value (e.g., 2 for red), ensuring no overflow
    agent_value = 2
    for x, y in agents:
        display_grid[x, y] = agent_value

    # Mark food sources with another value (e.g., 3 for white), ensuring no overflow
    food_value = 3
    for x, y in food_sources:
        display_grid[x, y] = food_value

    # Set custom color map
    from matplotlib.colors import ListedColormap
    custom_cmap = ListedColormap(['black', '#7D0000', '#FF0000', 'white'])  # Adjust gray shade as needed

    # Visualization of the grid with the custom color map
    axs[0].imshow(display_grid, cmap=custom_cmap, interpolation='nearest', vmin=0, vmax=3)
    axs[0].set_title(f"Agents & Food Sources (Step {step})")

    # Visualization of the odor map
    axs[1].imshow(odor_map, cmap='viridis', interpolation='nearest')
    axs[1].set_title("Odor Map")

    display(fig)
    clear_output(wait=True)

### Interactive Widget: run_simulation

#### Purpose
`run_simulation` provides an interactive interface for users to easily configure and run the Slime Mould Algorithm (SMA) simulation. It utilizes IPython widgets to create a user-friendly, interactive setup where parameters can be adjusted before running the simulation.

#### Parameters
- `num_agents`: The number of agents to include in the simulation. Adjustable through an integer slider.
- `num_food_sources`: The number of food sources to be placed on the grid. Adjustable through an integer slider.
- `grid_size`: The size of the square simulation grid. Adjustable through an integer slider.
- `steps`: The total number of steps the simulation will run. Adjustable through an integer slider.
- `A_param`: A parameter that influences the amplitude of oscillatory components in agent movement. Adjustable through a float slider.
- `start_in_middle`: A boolean checkbox to determine whether agents start in the middle of the grid or are randomly distributed.

#### Functionality
1. **Parameter Selection**: Allows users to interactively set the parameters of the simulation using sliders and checkboxes.
2. **Simulation Execution**: On user interaction, the function gathers the specified parameters and invokes the `simulate_SMA` function to run the simulation with the chosen settings.

#### User Interaction
- Users can adjust the sliders and checkbox to experiment with different simulation configurations.
- The simulation starts or updates when the user triggers the interaction, typically by a button press or slider adjustment.

This widget significantly enhances the usability and exploratory potential of the SMA simulation. It empowers users to experiment with different parameters, facilitating a deeper understanding of the simulation dynamics and the behavior of slime mould under various conditions.

In [99]:
# Interactive widget to run the simulation
@interact_manual
def run_simulation(num_agents=widgets.IntSlider(min=1, max=500, value=50, description='Agents:'),
                   num_food_sources=widgets.IntSlider(min=1, max=500, value=50, description='Food:'),
                   grid_size=widgets.IntSlider(min=10, max=250, value=100, description='Grid Size:'),
                   steps=widgets.IntSlider(min=10, max=200, value=100, description='Steps:'),
                   A_param=widgets.FloatSlider(min=0, max=2, value=1, description='A Parameter:'),
                   start_in_middle=widgets.Checkbox(value=False, description='Start in Middle')):
    simulate_SMA(grid_size, num_agents, num_food_sources, steps, A_param, start_in_middle)

interactive(children=(IntSlider(value=50, description='Agents:', max=500, min=1), IntSlider(value=50, descript…